In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
seed = 7
scoring = 'neg_mean_squared_error'

pisa = pd.read_csv('pisa2009.csv')
y = pisa['readingScore']
pisa = pisa.drop('readingScore', axis=1)

# Missing values
# For continuous variables replace the missing data with average of columns
pisa['minutesPerWeekEnglish'].fillna(int(pisa['minutesPerWeekEnglish'].mean()), inplace=True)
pisa['studentsInEnglish'].fillna(int(pisa['studentsInEnglish'].mean()), inplace=True)
pisa['schoolSize'].fillna(int(pisa['schoolSize'].mean()), inplace=True)

# For 'raceeth' use 'missing' replacement
pisa['raceeth'].fillna('NoRace', inplace=True)
# For binary variable use 0.5 in place of missing value

pisa = pisa.apply(lambda x:x.fillna(0.5))

# Encode categorical data
raceeth = pd.get_dummies(pisa['raceeth'], 'raceeth')
pisa = pd.concat([pisa, raceeth], axis=1)
pisa = pisa.drop('raceeth', axis=1)

# remove multicolinearity
pisa = pisa.drop(['motherBachelors','motherBornUS', 'fatherBornUS'], axis=1)

X = pisa

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [3]:

# Cross validation framework

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor


def do_cross_validation(X, y, scoring = 'neg_mean_squared_error'):
    print(X.shape)
    models = []
    models.append(('LR', LinearRegression()))
    models.append(('GB', GradientBoostingRegressor() ))
    
    results_rmse = []
    results_r2 = []
    names = []
    for name, model in models:
        kfold = KFold(n_splits=10, random_state=7)
        cv_results = np.sqrt(-cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring))
        cv_results_r2 = cross_val_score(model, X_train, y_train, cv=kfold, scoring='r2')
        results_rmse.append(cv_results)
        results_r2.append(cv_results_r2)
        names.append(name)
        msg = "%3s: %.3f (%.3f) r2: %.3f (%.3f)" % \
        (name, cv_results.mean(), cv_results.std(), cv_results_r2.mean(), cv_results_r2.std())
        print(msg)
    return results_rmse, results_r2, names

### Baseline before feature selection

In [4]:
rmse, r2, names = do_cross_validation(X_train, y_train)

(4186, 27)
 LR: 77.638 (2.122) r2: 0.341 (0.035)
 GB: 74.311 (2.130) r2: 0.396 (0.038)


## Feature selection 1. SelectKBest

In [5]:
from sklearn.feature_selection import SelectKBest
from numpy import set_printoptions
from sklearn.feature_selection import f_regression

# feature extraction

test = SelectKBest(score_func=f_regression, k=2)
fit = test.fit(X_train, y_train)
# summarize scores
set_printoptions(precision=3)
i = 0
KBest_ranking = {}
for score in fit.scores_:
    KBest_ranking[pisa.columns[i]] = score
    i +=1

for key in sorted(KBest_ranking, key=KBest_ranking.get, reverse=True):
    print "%3d %s" % (KBest_ranking[key], key)

645 expectBachelors
385 raceeth_White
354 grade
252 read30MinsADay
223 raceeth_Black
204 fatherHS
174 fatherBachelors
172 computerForSchoolwork
141 raceeth_Hispanic
116 motherHS
 94 male
 74 publicSchool
 74 englishAtHome
 40 fatherWork
 32 raceeth_Asian
 29 schoolHasLibrary
 19 urban
 17 selfBornUS
 17 minutesPerWeekEnglish
 14 raceeth_American Indian/Alaska Native
 13 preschool
 11 raceeth_NoRace
  8 motherWork
  6 studentsInEnglish
  3 schoolSize
  0 raceeth_More than one race
  0 raceeth_Native Hawaiian/Other Pacific Islander


In [6]:
features = fit.transform(X_train)
_ = do_cross_validation(features, y_train)

(4186L, 2L)
 LR: 77.638 (2.122) r2: 0.341 (0.035)
 GB: 74.314 (2.137) r2: 0.396 (0.037)


No improvement with selectKBest

### Feature selection 2. RFE

In [15]:
from sklearn.feature_selection import RFE
model = GradientBoostingRegressor()
rfe = RFE(model, 2)
fit = rfe.fit(X_train, y_train)
#print("Num Features: %d") % fit.n_features_
#print("Selected Features: %s") % fit.support_
#print("Feature Ranking: %s") % fit.ranking_
#print pisaTrain.columns[fit.support_]
RFE_ranking = {}
i = 0
for rank in fit.ranking_:
    RFE_ranking[pisa.columns[i]] = rank
    i += 1

for key in sorted(RFE_ranking, key=RFE_ranking.get):
    print "%2d %s" % (RFE_ranking[key], key) 

 1 minutesPerWeekEnglish
 1 schoolSize
 2 studentsInEnglish
 3 expectBachelors
 4 grade
 5 read30MinsADay
 6 raceeth_White
 7 fatherBachelors
 8 schoolHasLibrary
 9 computerForSchoolwork
10 publicSchool
11 raceeth_Black
12 male
13 englishAtHome
14 urban
15 raceeth_Asian
16 fatherHS
17 raceeth_More than one race
18 motherHS
19 motherWork
20 selfBornUS
21 fatherWork
22 preschool
23 raceeth_Native Hawaiian/Other Pacific Islander
24 raceeth_NoRace
25 raceeth_Hispanic
26 raceeth_American Indian/Alaska Native


In [16]:
selected = X_train[X_train.columns[fit.support_]]
_ = do_cross_validation(selected, y_train)

(4186, 2)
 LR: 77.638 (2.122) r2: 0.341 (0.035)
 GB: 74.311 (2.139) r2: 0.396 (0.038)


In [17]:
selected.shape

(4186, 2)

### Feature selection 2. Principal Component Analysis

In [18]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
fit = pca.fit(X_train)
# summarize components
print("Explained Variance: %s") %fit.explained_variance_ratio_
print(fit.components_)

Explained Variance: [  9.718e-01   2.814e-02   6.104e-05   6.315e-07]
[[  4.175e-05   3.480e-06  -1.389e-05   1.011e-05  -3.503e-05  -2.329e-05
   -3.322e-05   1.945e-05  -9.661e-06  -3.673e-05  -6.874e-05   2.595e-05
   -3.361e-06   1.405e-03   1.878e-03   5.220e-06   6.930e-05   1.772e-04
    1.000e+00  -6.855e-06   1.547e-05  -2.387e-06   1.122e-04  -2.363e-06
    1.704e-06  -1.078e-06  -1.167e-04]
 [  1.494e-04   1.725e-05  -3.725e-05   3.057e-05   5.046e-05   6.841e-05
    7.911e-05   1.940e-05   1.361e-05  -1.424e-05   2.895e-05  -2.095e-05
    6.755e-05   1.000e+00   1.777e-03  -2.390e-05   7.587e-05  -1.805e-04
   -1.408e-03  -4.276e-07   3.054e-06  -8.981e-05  -5.477e-05   3.340e-06
   -5.616e-06   9.229e-06   1.350e-04]
 [  6.414e-03  -2.787e-03  -1.619e-03   2.719e-03  -1.089e-03   6.070e-04
    1.420e-04  -2.132e-03   1.128e-03   4.860e-05  -1.178e-03   1.903e-03
   -1.168e-05  -1.780e-03   9.999e-01   1.364e-03   4.820e-04   5.974e-03
   -1.877e-03  -5.345e-04   1.581e-03 

In [19]:
Xpca = pca.transform(X_train)
_=do_cross_validation(Xpca, y_train)

(4186L, 4L)
 LR: 77.638 (2.122) r2: 0.341 (0.035)
 GB: 74.308 (2.131) r2: 0.396 (0.038)


### Feature selection 2. Feature Importance using ExtraTreesRegressor

In [12]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X_train, y_train)
#print(model.feature_importances_)
i = 0
res_dict = {}
for f in model.feature_importances_:
    res_dict[pisa.columns[i]] = f
    i += 1
    
#for key, value in sorted(res_dict.iteritems(), key=lambda(k,v): (v,k)):
#    print "%f %s" % (value, key)
featuresSorted = []
for key in sorted(res_dict, key=res_dict.get, reverse=True):
    print "%.4f %s" % (res_dict[key], key)
    featuresSorted.append(key)

0.1357 expectBachelors
0.1043 schoolSize
0.0946 minutesPerWeekEnglish
0.0921 studentsInEnglish
0.0731 grade
0.0700 raceeth_White
0.0490 fatherBachelors
0.0431 read30MinsADay
0.0340 motherWork
0.0310 urban
0.0302 male
0.0293 preschool
0.0265 fatherWork
0.0246 computerForSchoolwork
0.0240 fatherHS
0.0213 schoolHasLibrary
0.0194 selfBornUS
0.0188 motherHS
0.0180 englishAtHome
0.0172 raceeth_Asian
0.0167 publicSchool
0.0110 raceeth_Black
0.0057 raceeth_Hispanic
0.0054 raceeth_More than one race
0.0029 raceeth_Native Hawaiian/Other Pacific Islander
0.0012 raceeth_American Indian/Alaska Native
0.0007 raceeth_NoRace


In [13]:
#top 10 columns
featuresSorted[:5]

['expectBachelors',
 'schoolSize',
 'minutesPerWeekEnglish',
 'studentsInEnglish',
 'grade']

In [14]:
_ = do_cross_validation(X_train[featuresSorted[:1]], y_train)

(4186, 1)
 LR: 77.638 (2.122) r2: 0.341 (0.035)
 GB: 74.303 (2.130) r2: 0.396 (0.037)
